In [1]:
import pandas as pd
%load_ext autoreload
%autoreload 1

In [2]:
from src.pipeline import *
# from src.convert_graphml_to_pyg_data_multithread import *

optuna.logging.set_verbosity(optuna.logging.WARNING)

%aimport src.pipeline
# %aimport src.convert_graphml_to_pyg_data_multithread

In [3]:
features = [
    "faces", "edges", "vertices", "quantity",
    "height", "width", "depth", "volume", "area",
    "bbox_height", "bbox_width", "bbox_depth", "bbox_volume",
    "bbox_area",
]
data = pd.read_csv("./data/synced_dataset_final.csv")

In [10]:
X = data[features]
X_train, X_test, y_train_index, y_test_index = train_test_split(
    X, range(len(X)),
    test_size=0.2,
    random_state=100,
    stratify=data["is_cnc"])
y_multi_train = data["multiclass_labels"].iloc[y_train_index]
y_multi_test = data["multiclass_labels"].iloc[y_test_index]

y_binary_train = data["is_cnc"].iloc[y_train_index]
y_binary_test = data["is_cnc"].iloc[y_test_index]

params = {
    "n_estimators": randint(100, 300),
    "max_depth": randint(3, 10),
    "learning_rate": uniform(0.005, 0.3),
}
best_search_binary = randomizedsearchcv_xgboost(
    X_train, y_binary_train,
    params, n_iter=50,
)

In [11]:
y_binary_pred = best_search_binary.best_estimator_.predict(X_test)
y_binary_prob = best_search_binary.best_estimator_.predict_proba(X_test)
y_binary_pred_train = best_search_binary.best_estimator_.predict(X_train)
y_binary_prob_train = best_search_binary.best_estimator_.predict_proba(X_train)
metrics_binary_validation = evaluate_classification(
    y_true=y_binary_test,
    y_pred=y_binary_pred,
    y_prob=y_binary_prob,
)
metrics_binary_train = evaluate_classification(
    y_true=y_binary_train,
    y_pred=y_binary_pred_train,
    y_prob=y_binary_prob_train,
)
pd.DataFrame([metrics_binary_train, metrics_binary_validation], index=["train", "validation"]).T

,train,validation
accuracy,0.951304,0.870498
f1_macro,0.947830,0.860209
f1_weighted,0.951724,0.871178
roc_auc,0.992262,0.936981
average_precision,0.995679,0.961874


In [17]:
fold_results = xgboost_optuna(
    data,
    n_trials=2000,
    features=features
)

Optimizing fold 0.0:


  0%|          | 0/2000 [00:00<?, ?it/s]

Optimizing fold 1.0:


  0%|          | 0/2000 [00:00<?, ?it/s]

Optimizing fold 2.0:


...

  0%|          | 0/2000 [00:00<?, ?it/s]

Optimizing fold 8.0:


  0%|          | 0/2000 [00:00<?, ?it/s]

Optimizing fold 9.0:


  0%|          | 0/2000 [00:00<?, ?it/s]

In [18]:
with open(r"./data/xgboost_binary_optuna_results.pkl", "wb") as f:
    joblib.dump(fold_results, f)

In [24]:
pd.DataFrame({key: (value["study"].best_trial.user_attrs["train_auroc"],
                    value["best_score"]) for key, value in
              fold_results.items()}, index=["train_auroc", "validation_auroc"]).T

,train_auroc,validation_auroc
fold_0.0,0.993062,0.946157
fold_1.0,0.992681,0.944985
fold_2.0,0.992461,0.943859
fold_3.0,0.992555,0.941675
fold_4.0,0.993113,0.941118
fold_5.0,0.992562,0.943324
fold_6.0,0.992182,0.940895
fold_7.0,0.993285,0.940183
fold_8.0,0.993506,0.940808
fold_9.0,0.993249,0.942123


In [25]:
auroc_scores = [item["best_score"] for item in fold_results.values()]
average_mean = np.mean(auroc_scores)
average_std = np.std(auroc_scores)
print(f"Average AUROC: {average_mean:.6f} ± {average_std:.6f}")

Average AUROC: 0.942513 ± 0.001887
